In [5]:
import os
import glob
import json
import torch
import numpy as np

# Anzahl der Klassen (Cityscapes)
NUM_CLASSES = 20

# Dictionary mit Klassennamen
city_classes = {
    0: 'road',
    1: 'sidewalk',
    2: 'building',
    3: 'wall',
    4: 'fence',
    5: 'pole',
    6: 'traffic light',
    7: 'traffic sign',
    8: 'vegetation',
    9: 'terrain',
    10: 'sky',
    11: 'person',
    12: 'rider',
    13: 'car',
    14: 'truck',
    15: 'bus',
    16: 'train',
    17: 'motorcycle',
    18: 'bicycle',
    19: 'unlabeled',
}

# Basisverzeichnis
BASE_PATH = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation"
# Ordner, in dem die Confusion Matrices für K-Fold liegen (z.B. Fold0, Fold1, ...)
CONF_MATRIX_BASE = os.path.join(BASE_PATH, "FINAL_DATEN", "confusion_matrices_k_fold")

# Dateiname für die JSON-Ausgabe
OUTPUT_JSON = os.path.join(BASE_PATH, "FINAL_DATEN", "evaluation_results_k_fold.json")

print("CONF_MATRIX_BASE:", CONF_MATRIX_BASE)
print("OUTPUT_JSON:", OUTPUT_JSON)

CONF_MATRIX_BASE: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_k_fold
OUTPUT_JSON: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/evaluation_results_k_fold.json


In [6]:
def compute_confusion_matrix_metrics(cm):
    """
    Berechnet Mean IoU, FWIoU und pro-Klassen IoU, sowie die relative Häufigkeit jeder Klasse.
    Gibt ein Dictionary mit den wichtigsten Werten zurück.
    """
    # Intersection, Union, etc.
    intersection = torch.diag(cm).float()
    ground_truth_sum = cm.sum(1).float()
    predicted_sum = cm.sum(0).float()
    union = ground_truth_sum + predicted_sum - intersection + 1e-6
    
    iou_per_class = intersection / union
    mIoU = torch.mean(iou_per_class).item()
    
    # Frequency Weighted IoU
    total_pixels = cm.sum().float() + 1e-6
    fwiou = (ground_truth_sum * iou_per_class).sum().item() / total_pixels.item()
    
    # Relative Häufigkeit (Frequency in %)
    frequency_percent = (ground_truth_sum / total_pixels) * 100.0
    
    return {
        "mIoU": mIoU,
        "FWIoU": fwiou,
        "iou_per_class": iou_per_class,
        "frequency_percent": frequency_percent
    }

In [7]:
NUM_FOLDS = 5  # Wir gehen davon aus, dass es 5 Folds (0..4) gibt
results_summary = {}  # Speichert die Metriken pro Fold und Modell
miou_std_summary = {}  # Speichert die mIoU-Standardabweichungen pro Modell
fwiou_std_summary = {}  # Speichert die FWIoU-Standardabweichungen pro Modell
miou_avg_summary = {}  # Speichert den durchschnittlichen mIoU pro Modell
fwiou_avg_summary = {}  # Speichert den durchschnittlichen FWIoU pro Modell

# Daten sammeln
for fold in range(NUM_FOLDS):
    fold_dir = os.path.join(CONF_MATRIX_BASE, f"Fold{fold}")
    if not os.path.isdir(fold_dir):
        print(f"[WARNING] Ordner {fold_dir} existiert nicht. Überspringe Fold {fold}.")
        continue
    
    cm_files = glob.glob(os.path.join(fold_dir, "*_confusion_matrix.pt"))
    print(f"\n=== Fold {fold}: {len(cm_files)} Confusion Matrices gefunden ===")
    
    fold_results = {}
    
    for cm_file in cm_files:
        base_name = os.path.basename(cm_file)
        model_name = base_name.replace("_confusion_matrix.pt", "")
        
        cm = torch.load(cm_file)
        metrics = compute_confusion_matrix_metrics(cm)
        
        fold_results[model_name] = {
            "mIoU": metrics["mIoU"],
            "FWIoU": metrics["FWIoU"],
            "iou_per_class": metrics["iou_per_class"].cpu().tolist(),
            "frequency_percent": metrics["frequency_percent"].cpu().tolist(),
        }
        
        print(f"\nModell: {model_name}")
        print(f"Mean IoU: {metrics['mIoU']:.4f}")
        print(f"FWIoU: {metrics['FWIoU']:.4f}")
        
        print("\nPer-Klassen Metriken:")
        print("{:<12} {:<40} {:>10} {:>15}".format("Class Index", "Class Name", "IoU", "Frequency (%)"))
        print("-"*70)
        
        iou_values = metrics["iou_per_class"]
        freq_values = metrics["frequency_percent"]
        
        for i in range(NUM_CLASSES):
            class_name = city_classes.get(i, f"Class {i}")
            print("{:<12} {:<40} {:>10.4f} {:>15.2f}".format(
                i,
                class_name,
                iou_values[i].item(),
                freq_values[i].item()
            ))
    
    results_summary[f"Fold{fold}"] = fold_results

# Berechne Standardabweichung und Durchschnitt über alle Folds
all_models = set()
for fold_data in results_summary.values():
    all_models.update(fold_data.keys())

for model_name in all_models:
    miou_list = []
    fwiou_list = []
    
    for fold_data in results_summary.values():
        if model_name in fold_data:
            miou_list.append(fold_data[model_name]["mIoU"])
            fwiou_list.append(fold_data[model_name]["FWIoU"])
    
    if miou_list:
        miou_std_summary[model_name] = float(np.std(miou_list))
        fwiou_std_summary[model_name] = float(np.std(fwiou_list))
        miou_avg_summary[model_name] = float(np.mean(miou_list))
        fwiou_avg_summary[model_name] = float(np.mean(fwiou_list))

# Kombiniere alles zu einem JSON-Dokument
output_json_data = {
    "folds": results_summary,
    "summary": {
        model: {
            "avg_mIoU": miou_avg_summary.get(model, None),
            "std_mIoU": miou_std_summary.get(model, None),
            "avg_FWIoU": fwiou_avg_summary.get(model, None),
            "std_FWIoU": fwiou_std_summary.get(model, None)
        }
        for model in all_models
    }
}

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(output_json_data, f, indent=4)
print(f"\nAlle Ergebnisse inkl. Standardabweichung und Durchschnitt wurden in JSON gespeichert: {OUTPUT_JSON}")



=== Fold 0: 6 Confusion Matrices gefunden ===

Modell: fcn_resnet101
Mean IoU: 0.5201
FWIoU: 0.8013

Per-Klassen Metriken:
Class Index  Class Name                                      IoU   Frequency (%)
----------------------------------------------------------------------
0            road                                         0.9256           32.62
1            sidewalk                                     0.6943            5.12
2            building                                     0.8120           19.85
3            wall                                         0.2391            0.50
4            fence                                        0.3728            0.67
5            pole                                         0.2202            1.12
6            traffic light                                0.3833            0.18
7            traffic sign                                 0.5287            0.50
8            vegetation                                   0.8465           1

In [8]:
# Sammle alle Modelle, die in irgendeinem Fold vorkommen
all_models = set()
for fold_name in results_summary:
    for model_name in results_summary[fold_name]:
        all_models.add(model_name)

# Für jedes gefundene Modell wird nun eine Tabelle ausgegeben
for model_name in sorted(all_models):
    # Sammle pro Fold die iou_per_class, frequency_percent, mIoU und FWIoU
    fold_data = {}
    miou_list = []
    fwiou_list = []
    for fold_name in results_summary:
        if model_name in results_summary[fold_name]:
            iou_list = results_summary[fold_name][model_name]["iou_per_class"]
            freq_list = results_summary[fold_name][model_name]["frequency_percent"]
            miou = results_summary[fold_name][model_name]["mIoU"]
            fwiou = results_summary[fold_name][model_name]["FWIoU"]
            fold_data[fold_name] = (iou_list, freq_list)
            miou_list.append(miou)
            fwiou_list.append(fwiou)
        else:
            fold_data[fold_name] = (None, None)
    
    # Berechne Standardabweichung für mIoU und FWIoU über alle Folds
    miou_std = np.std(miou_list) if miou_list else None
    fwiou_std = np.std(fwiou_list) if fwiou_list else None

    # Kopfzeile
    print(f"\n{'='*70}")
    print(f"Modell: {model_name}")
    print(f"{'='*70}")
    if miou_std is not None and fwiou_std is not None:
        print(f"mIoU_std: {miou_std:.4f}   FWIoU_std: {fwiou_std:.4f}")
    else:
        print("mIoU_std: -   FWIoU_std: -")
    
    # Baue eine Kopfzeile mit den Spalten: Class_ID, Class_Name, dann je Fold -> IoU, Freq
    header = f"{'Class_ID':<10} {'Class_Name':<25}"
    sorted_folds = sorted(fold_data.keys())  # z.B. Fold0, Fold1, ...
    for fold_name in sorted_folds:
        header += f"{fold_name}_IoU   {fold_name}_Freq   "
    print(header)
    print("-"*70)

    # Zeilen pro Klasse
    for cls_idx in range(NUM_CLASSES):
        class_name = city_classes.get(cls_idx, f"Class {cls_idx}")
        row_str = f"{cls_idx:<10} {class_name:<25}"

        # Für jeden Fold die IoU und Frequency einfügen
        for fold_name in sorted_folds:
            iou_freq = fold_data[fold_name]
            if iou_freq[0] is not None:
                iou_val = iou_freq[0][cls_idx]
                freq_val = iou_freq[1][cls_idx]
                row_str += f"{iou_val:>9.4f} {freq_val:>12.2f}   "
            else:
                row_str += f"{'-':>9} {'-':>12}   "

        print(row_str)



Modell: deeplabv3_mobilenet_v3_large
mIoU_std: 0.0111   FWIoU_std: 0.0016
Class_ID   Class_Name               Fold0_IoU   Fold0_Freq   Fold1_IoU   Fold1_Freq   Fold2_IoU   Fold2_Freq   Fold3_IoU   Fold3_Freq   Fold4_IoU   Fold4_Freq   
----------------------------------------------------------------------
0          road                        0.9151        32.62      0.9182        32.65      0.9143        32.23      0.9199        32.90      0.9194        33.00   
1          sidewalk                    0.6596         5.12      0.6492         5.30      0.6521         5.57      0.6312         5.02      0.6336         5.42   
2          building                    0.7995        19.85      0.7990        19.73      0.8039        20.75      0.7961        19.92      0.7972        19.97   
3          wall                        0.2615         0.50      0.3782         0.65      0.2737         0.49      0.2710         0.56      0.4106         0.74   
4          fence                       0.375